# Tutorial: Menangani Nilai Yang Hilang Dengan Scikit Learn

nilai yang hilang (NaN) adalah musuh utama dalam persiapan data, sebagaian besar algoritma machine learning tidak dapat memprosesnya dan akan berhenti dengan error. di notebook ini, kita akan belajar startegi-strategi untuk menangani NaN secara sistematis menggunakan SimpleImputer dari Scikit-Learn

## 1. Mengapa Scikit-learn? Mengapa tidak hanya .fillna()?

Anda mungkin sudah familiar dengan metode .fillna() dari Pandas. Meskipun berguna untuk analisis cepat, pendekatan Scikit-learn lebih unggul untuk alur kerja machine learning karena:

* Mencegah Data Leakage: Scikit-learn memungkinkan kita untuk “belajar” parameter (seperti nilai rata-rata) hanya dari training set dan kemudian menerapkannya ke testing set. Ini adalah praktik yang benar secara metodologis.
* Integrasi Pipeline: Objek imputer dari Scikit-learn dapat dengan mudah diintegrasikan ke dalam Pipeline dan ColumnTransformer, membuat alur kerja persiapan data Anda bersih dan dapat direproduksi.

### Langkah 1: Identifikasi Masalah

sebelum kita bisa memperbaiki apa pun. kita harus tau dimana masalahnya, gunakan metode isnull().sum() pada DataFrame untuk mendapatkan hitungan nilai yang hilang di setiap kolom

In [3]:
import numpy as np
import pandas as pd

In [12]:
data = {
    'Usia': [25,28,np.nan,35,32,np.nan],
    'Gaji': [50000,54000,52000,np.nan,60000,620000],
    'Jenis Kelamin': ['Pria', 'Wanita', 'Wanita', 'Pria', np.nan, 'Pria'],
}
df = pd.DataFrame(data)

In [15]:
# mengidentifikasi data yang hilang
print('Data Awal')
print(df)

Data Awal
   Usia      Gaji Jenis Kelamin
0  25.0   50000.0          Pria
1  28.0   54000.0        Wanita
2   NaN   52000.0        Wanita
3  35.0       NaN          Pria
4  32.0   60000.0           NaN
5   NaN  620000.0          Pria


In [18]:
print('\nJumlah nilai  hilang per kolom')
print(df.isnull().sum())


Jumlah nilai  hilang per kolom
Usia             2
Gaji             1
Jenis Kelamin    1
dtype: int64


### Langkah 2: menginputasi data yang hilang

`SimpleImputer` dari scikit-learn menawarkan beberapa startegi untu mengisi nilai yang hilang

| Strategi        | Deskripsi                                                                 | Kapan Digunakan                                                                 |
|-----------------|---------------------------------------------------------------------------|---------------------------------------------------------------------------------|
| `mean`          | Mengisi dengan nilai **rata-rata** dari kolom tersebut.                  | Pilihan default yang baik untuk fitur numerik yang terdistribusi normal (tanpa *outlier* ekstrem). |
| `median`        | Mengisi dengan nilai **tengah** dari kolom tersebut.                     | Pilihan yang lebih aman (*robust*) untuk fitur numerik yang memiliki *outlier* atau distribusinya miring (*skewed*). |
| `most_frequent` | Mengisi dengan nilai yang **paling sering muncul** (modus).             | Pilihan utama untuk mengisi data **kategorikal**.                               |
| `constant`      | Mengisi dengan nilai konstan yang Anda tentukan (misalnya `0` atau `"Tidak Diketahui"`). | Berguna jika ketiadaan data itu sendiri merupakan sebuah informasi.             |


### Langkah 3: Implementasi Dengan `SimpleImputer` (Data Numerik)

Mari kita terapkan ini dalam alur kerja machine learning yang benar, lengkap dengan train-test split

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [24]:
# kita fokus pada fitur numerik dulu
df_numeric = df[['Usia', 'Gaji']]
print(df_numeric)

   Usia      Gaji
0  25.0   50000.0
1  28.0   54000.0
2   NaN   52000.0
3  35.0       NaN
4  32.0   60000.0
5   NaN  620000.0


In [48]:
# 1. Lakukan Train Test Split
X_train, X_test = train_test_split(df_numeric,test_size=0.2,random_state=42)

print('----- Data Sebelum Imputasi -----')
print(f'X_train = \n{X_train}')
print(f'X_test = \n{X_test}')

print(X_train.columns)
print(X_train.index)

----- Data Sebelum Imputasi -----
X_train = 
   Usia      Gaji
5   NaN  620000.0
2   NaN   52000.0
4  32.0   60000.0
3  35.0       NaN
X_test = 
   Usia     Gaji
0  25.0  50000.0
1  28.0  54000.0
Index(['Usia', 'Gaji'], dtype='object')
Index([5, 2, 4, 3], dtype='int64')


In [46]:
#2. Inisialisasi Imputer
imputer = SimpleImputer(strategy='median')

# 3. Fit imputer hanya pada data training
print('\nMelatih imputer pada X_train')
imputer.fit(X_train)
print(
    "Nilai median yang dipelajari (Usia, Gaji):",
    [f"{x:g}" for x in imputer.statistics_]
)


Melatih imputer pada X_train
Nilai median yang dipelajari (Usia, Gaji): ['33.5', '60000']


In [36]:
# 4. transformasi data pelatihan dan pengujian
# menggunakan median dari data X_train untuk mengisi NaN
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)

[[3.35e+01 6.20e+05]
 [3.35e+01 5.20e+04]
 [3.20e+01 6.00e+04]
 [3.50e+01 6.00e+04]]
[[2.5e+01 5.0e+04]
 [2.8e+01 5.4e+04]]


In [38]:
# hasil menggunakan array numpy
X_train_imputed_df = pd.DataFrame(
    X_train_imputed,
    columns=X_train.columns,
    index=X_train.index
)
X_test_imputed_df = pd.DataFrame(
    X_test_imputed,
    columns=X_test.columns,
    index=X_test.index
)

print('----- Data Setelah Imputasi -----')
print(f'X_train_imputed_df = \n{X_train_imputed_df}')
print(f'X_test_imputed_df = \n{X_test_imputed_df}')

----- Data Setelah Imputasi -----
X_train_imputed_df = 
   Usia      Gaji
5  33.5  620000.0
2  33.5   52000.0
4  32.0   60000.0
3  35.0   60000.0
X_test_imputed_df = 
   Usia     Gaji
0  25.0  50000.0
1  28.0  54000.0


In [39]:
# 5. Menangani data kategorikal
df_categorical = df[['Jenis Kelamin']]
X_train_cat, X_test_cat = train_test_split(df_categorical,test_size=0.2,random_state=42)

print('----- Data Kategorikal Sebelum Imputasi -----')
print(f'X_train_cat: \n{X_train_cat}')
print(f'X_test_cat: \n{X_test_cat}')

----- Data Kategorikal Sebelum Imputasi -----
X_train_cat: 
  Jenis Kelamin
5          Pria
2        Wanita
4           NaN
3          Pria
X_test_cat: 
  Jenis Kelamin
0          Pria
1        Wanita


In [43]:
cat_imputer = SimpleImputer(strategy='most_frequent')

cat_imputer.fit(X_train_cat)
print("\nNilai paling sering dipelajari:", cat_imputer.statistics_)

X_train_cat_imputed = cat_imputer.transform(X_train_cat)
X_test_cat_imputed = cat_imputer.transform(X_test_cat)


Nilai paling sering dipelajari: ['Pria']


In [44]:
# hasil menggunakan array numpy
X_train_cat_imputed_df = pd.DataFrame(
    X_train_cat_imputed,
    columns=X_train_cat.columns,
    index=X_train_cat.index
)
X_test_cat_imputed_df = pd.DataFrame(
    X_test_cat_imputed,
    columns=X_test_cat.columns,
    index=X_test_cat.index
)

print('----- Data Setelah Imputasi -----')
print(f'X_train_cat_imputed_df = \n{X_train_cat_imputed_df}')
print(f'X_test_cat_imputed_df = \n{X_test_cat_imputed_df}')

----- Data Setelah Imputasi -----
X_train_cat_imputed_df = 
  Jenis Kelamin
5          Pria
2        Wanita
4          Pria
3          Pria
X_test_cat_imputed_df = 
  Jenis Kelamin
0          Pria
1        Wanita
